In [1]:
import json

In [3]:
from pathlib import Path

data = {}
for fn in Path("d").glob("*.json"):
    with open(fn) as f:
        try:
            data[fn.stem] = json.load(f)
        except json.JSONDecodeError as e:
            print(fn, e)
            
data["any-machines"] = {**data["assembling-machine"], **data["furnace"]}

data["any-items"] = {**data["item"], **data["fluid"]}

list(data.keys())

KeyError: 'assembling-machine'

In [3]:
from collections import defaultdict



def get_values(d):
    return {k: v.x for k, v in d.items() if v.x > 0}

def solve(model: mip.Model, source, target, stocks, coefs={"polution": 1, "electricity": 1}):
    produced = {i['name']: model.add_var(f"produced_{i['name']}", lb=0) for i in data["any-items"].values()}
    used = {i['name']: model.add_var(f"used_{i['name']}", lb=0) for i in data["any-items"].values()}

    electricity = []

    polution = []
    
    machines = {
        (
            r["name"],
            m["name"]
        ): model.add_var(f"machines_{m['name']}_{r['name']}", lb=0, var_type=mip.INTEGER)
        
        for r in data["recipe"].values()
        for m in data["any-machines"].values()
        if r["category"] in m['crafting_categories']
    }
    
    
    for i in data["any-items"].values():
        model.add_constr(produced[i['name']] - used[i['name']] <= target[i['name']] - source[i['name']], name=f"equilibrium_{i['name']}")
    
    for i in data["any-items"].values():
        right_hand_side_product = []
        right_hand_side_ingredident = []
        for r in data["recipe"].values():
            for m in data["any-machines"].values():
                
                if r["category"] in m['crafting_categories']:
                    for p in r['products']:
                        if p["name"] == i["name"]:
                            right_hand_side_product.append(machines[r["name"], m["name"]] * p['probability'] * p['amount'] * m["crafting_speed"])
                    
                    for p in r['ingredients']:
                        if p["name"] == i["name"]:
                            right_hand_side_ingredident.append(machines[r["name"], m["name"]] * p['amount'] * m["crafting_speed"])
        
        model.add_constr(mip.quicksum(right_hand_side_product) == produced[i['name']])
        model.add_constr(mip.quicksum(right_hand_side_ingredident) == used[i['name']])
    
    
    capital = {
        m["name"]: model.add_var(f"nb_machines_{m['name']}")
        for m in data["any-machines"].values()
    }
    
    electricity = mip.quicksum(capital[m['name']] * m["energy_usage"] for m in data["any-machines"].values())
    polution = mip.quicksum(capital[m['name']] * m["pollution"] for m in data["any-machines"].values())
    
    model.objective = electricity * coefs["electricity"] - polution * coefs["polution"]
    
    for m in data["any-machines"].values():
        model.add_constr(capital[m['name']] == mip.quicksum(machines[r['name'], m['name']] for r in data["recipe"].values() if r["category"] in m['crafting_categories']))
    
    
    
    model.optimize()
    
    print(model.status)

    
    return tuple(list(map(get_values, [produced, used, machines, capital])) + [electricity.x, polution.x])


In [4]:
from pprint import pprint
_, _, machines, _, _, _ = solve(mip.Model(), defaultdict(lambda: 0, {"iron-ore": 50, "copper-ore": 50}), defaultdict(lambda: 0, {"automation-science-pack": 100}), defaultdict(lambda: float('+inf')))

for (r, m), n in machines.items():
    r = data["recipe"][r]
    m = data["any-machines"][m]
    print(m["name"], int(n), len(r["ingredients"]))

OptimizationStatus.OPTIMAL
crash-site-assembling-machine-2-repaired 50 2
electric-furnace 25 1
electric-furnace 25 1
crash-site-assembling-machine-2-repaired 25 1


In [5]:
import pydantic
from typing import *
from enum import Enum

import nest_asyncio
import uvicorn



# data["any-items"].keys()

class Coefs(str, Enum):
    polution = "polution"
    electricity = "electricity"

class StrEnum(str, Enum):
    pass

Items = StrEnum("Item", {k.replace("-", "_"): k for k, _ in data["any-items"].items()})
Machines = StrEnum("Machines", {k.replace("-", "_"): k for k, _ in data["any-machines"].items()})


import fastapi

app = fastapi.FastAPI()

@app.post("/recipe-optimizer")
async def solve_recipe(source: Dict[Items, float], target: Dict[Items, float], stocks: Dict[Machines, int], coefs: Dict[Coefs, float]):
    source = defaultdict(lambda: 0, source)
    target = defaultdict(lambda: 0, target)
    stocks = defaultdict(lambda: float("+inf"), stocks)
    coefs = defaultdict(lambda: 1, coefs)
    
    _, _, machines, _, _, _ = solve(mip.Model(), source, target, stocks, coefs)
    
    return machines
    



nest_asyncio.apply()
uvicorn.run(app, port=8000)

INFO:     Started server process [5547]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [5547]
